# 🖼️ LLaVA 多模态指令微调数据构造流水线
本 Notebook 整合了从 **COCO 数据对齐** 到 **多模态大模型(Qwen-VL) 自动标注** 的完整流程。适用于构建 LLaVA、Qwen-VL 等模型的训练数据集。

### 1. 环境准备
安装必要的依赖包。

In [ ]:
pip install opencv-python tqdm openai numpy

### 2. 第一阶段：精准坐标对齐 (COCO Format to LLaVA)
将标准的 COCO 检测框 `[x, y, w, h]` 转换为 LLaVA 格式的归一化坐标 `[ymin, xmin, ymax, xmax]` (0-1000)。

In [ ]:
import os, json, glob, cv2
from tqdm.notebook import tqdm

IMAGE_DIR = "../data/images/"
ANNOTATION_FILE = "../data/annotations/instances_val2017.json"
OUTPUT_ALIGN_FILE = "../data/llava_instruct_aligned.json"

def convert_bbox(bbox, width, height):
    x, y, w, h = bbox
    xmin = int((x / width) * 1000)
    ymin = int((y / height) * 1000)
    xmax = int((x + w) / width * 1000)
    ymax = int((y + h) / height * 1000)
    return [max(0, min(1000, ymin)), max(0, min(1000, xmin)), 
            max(0, min(1000, ymax)), max(0, min(1000, xmax))]

def load_coco():
    with open(ANNOTATION_FILE, 'r') as f: coco = json.load(f)
    img_to_anns = {}
    cat_map = {cat['id']: cat['name'] for cat in coco['categories']}
    for ann in coco['annotations']:
        img_id = ann['image_id']
        if img_id not in img_to_anns: img_to_anns[img_id] = []
        img_to_anns[img_id].append({"bbox": ann['bbox'], "label": cat_map.get(ann['category_id'], "object")})
    return img_to_anns

if os.path.exists(ANNOTATION_FILE):
    img_to_anns = load_coco()
    dataset = []
    image_paths = glob.glob(os.path.join(IMAGE_DIR, "*.jpg"))
    
    for img_path in tqdm(image_paths, desc="对齐数据"):
        fname = os.path.basename(img_path)
        try: image_id = int(fname.split('.')[0])
        except: continue
        anns = img_to_anns.get(image_id, [])
        img = cv2.imread(img_path)
        if img is None: continue
        h, w, _ = img.shape
        for ann in anns[:3]: # 每张图取3个物体
            box = convert_bbox(ann['bbox'], w, h)
            dataset.append({
                "id": f"{image_id}_{ann['label']}",
                "image": fname,
                "conversations": [
                    {"from": "human", "value": f"Where is the {ann['label']}? <image>"},
                    {"from": "qwen", "value": f"The {ann['label']} is at {box}."}
                ]
            })
    with open(OUTPUT_ALIGN_FILE, 'w') as f: json.dump(dataset, f, indent=2)
    print(f"对齐完成，保存至 {OUTPUT_ALIGN_FILE}")

### 3. 第二阶段：LLM 增强标注 (Cognitive Data Generation)
利用 Qwen2.5-VL 等大模型生成包含推理、细节描述的对话。

In [ ]:
import base64
from openai import OpenAI

API_KEY = "sk-lrdpxzsnhsbckhjrzekbrtccomruhcwzyrlwbroqwojtwtsw"
client = OpenAI(api_key=API_KEY, base_url="https://api.siliconflow.cn/v1")

def encode_img(p): 
    with open(p, "rb") as f: return base64.b64encode(f.read()).decode('utf-8')

def generate_llava_json(img_path):
    b64 = encode_img(img_path)
    response = client.chat.completions.create(
        model="Qwen/Qwen2.5-VL-72B-Instruct",
        messages=[{"role": "system", "content": "直接返回JSON，包含两轮对话：描述图片和细节提问。使用[ymin, xmin, ymax, xmax]格式。"},
                  {"role": "user", "content": [{"type":"text","text":"分析图片"}, 
                                               {"type":"image_url","image_url":{"url":f"data:image/jpeg;base64,{b64}"}}]}]
    )
    return response.choices[0].message.content

### 4. 第三阶段：多图对比数据 (Interleaved Data)
构造两张图片交错输入的对话，用于训练模型对比和逻辑推理能力。

In [ ]:
import random
def generate_interleaved(img1, img2):
    prompt = "Compare these two images."
    # 调用模型并构造 LLaVA Interleaved 格式...
    # 结果示例如下：
    return {
        "id": "compare_001",
        "image": [os.path.basename(img1), os.path.basename(img2)],
        "conversations": [
            {"from": "human", "value": "Image 1: <image>\nImage 2: <image>\nWhat is the difference?"},
            {"from": "qwen", "value": "The first image is a city, the second is a park."}
        ]
    }

### 5. 第四阶段：可视化验证 (Debug Visualization)
读取生成的 JSON 文件，将坐标框反向画在图片上，检查 LLM 输出的坐标是否精准。

In [ ]:
import re
VIZ_OUT = "../data/viz_debug/"
os.makedirs(VIZ_OUT, exist_ok=True)

with open(OUTPUT_ALIGN_FILE, 'r') as f: data = json.load(f)

for entry in data[:5]: # 抽样5张验证
    img = cv2.imread(os.path.join(IMAGE_DIR, entry['image']))
    h, w, _ = img.shape
    for turn in entry['conversations']:
        bboxes = re.findall(r'\[(\d+),\s*(\d+),\s*(\d+),\s*(\d+)\]', turn['value'])
        for b in bboxes:
            ymin, xmin, ymax, xmax = [int(x) for x in b]
            cv2.rectangle(img, (int(xmin*w/1000), int(ymin*h/1000)), 
                          (int(xmax*w/1000), int(ymax*h/1000)), (0, 255, 0), 2)
    cv2.imwrite(os.path.join(VIZ_OUT, f"viz_{entry['image']}"), img)
print(f"验证图片已保存至 {VIZ_OUT}")